In [33]:
%matplotlib notebook
import sys
from importlib import reload
import pandas as pd
import numpy as np
import csv

from random import randint

reload(sys)

from sklearn.linear_model import LogisticRegression
import xgboost as xgbq
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import xgboost as xgb


from tqdm import tqdm
tqdm.pandas()

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [65]:
frame = pd.read_csv(r'/Users/Milton/Developer/Hockey/Article Testing Sandbox/Favorites_Full_Dataset_Games_Standings_2007-2018.csv', index_col=0)

# Remove games played in 2017-2018 season and use that dataset to test instead
import datetime
    
def selectGamesFromNewSeason(row):
    currentDate = datetime.datetime.strptime(row, '%Y/%m/%d')
    startDate = datetime.datetime(2017, 9, 1)
    endDate = datetime.datetime(2018, 5, 1)
    
    if startDate <= currentDate <= endDate:
        return True
    else: 
        return False

frame.loc[frame['Result'] == 'H', 'Result'] = 1
frame.loc[frame['Result'] == 'A', 'Result'] = 0

newGamesFrame = frame[frame['Date'].apply(lambda row: selectGamesFromNewSeason(row))]
frame = pd.concat([frame, newGamesFrame]).drop_duplicates(keep=False)
display(frame.loc[frame['Date'].str.contains('2018')])

newx_all = newGamesFrame.drop(['Result', 'ID', 'Date', 'HomeTeam', 'AwayTeam'], axis=1)
newy_all = newGamesFrame['Result']

x_all = frame.drop(['Result', 'ID', 'Date', 'HomeTeam', 'AwayTeam'], axis=1)
y_all = frame['Result']

# Standardizing data
from sklearn.preprocessing import scale

columns = x_all.columns.values

for column in columns:
    x_all[column] = scale(x_all[column])
    
newColumns = x_all.columns.values

for newColumn in newColumns:
    newx_all[newColumn] = scale(newx_all[newColumn])
    
display(x_all.head(1))

,ID,Date,HomeTeam,AwayTeam,CF%,CSh%,CSv%,FF%,FSh%,FSv%,...,PDO,PENDiff,SF%,SDiff,Sh%,Sv,FOW%,W%,FavoritesW%,Result


,CF%,CSh%,CSv%,FF%,FSh%,FSv%,GDiff,GF%,PDO,PENDiff,SF%,SDiff,Sh%,Sv,FOW%,W%,FavoritesW%
181,1.040103,-1.678695,-0.404192,0.890108,-1.827343,-0.067838,-0.456788,-0.491924,-1.354853,1.010538,0.975585,1.029864,-2.137419,0.066379,0.348422,-0.128719,0.691512


In [35]:
frame

,ID,Date,HomeTeam,AwayTeam,CF%,CSh%,CSv%,FF%,FSh%,FSv%,...,PDO,PENDiff,SF%,SDiff,Sh%,Sv,FOW%,W%,FavoritesW%,Result
181,https://www.nhl.com/gamecenter/2007020182,2007/11/02,New Jersey Devils,Toronto Maple Leafs,4.42,-1.413540,-0.354732,3.76,-1.988645,-0.079722,...,-3.05,31.0,4.12,4.97,-3.151893,0.10,1.1,-0.020979,0.350000,1
182,https://www.nhl.com/gamecenter/2007020183,2007/11/02,Washington Capitals,Philadelphia Flyers,10.24,-2.807782,-1.290578,7.69,-3.223536,-1.473762,...,-6.10,7.0,6.62,7.90,-4.191705,-1.91,3.3,-0.245455,0.420000,0
183,https://www.nhl.com/gamecenter/2007020184,2007/11/02,Buffalo Sabres,Florida Panthers,-0.13,2.300800,-1.221793,0.94,2.814914,-1.779103,...,1.21,-7.0,1.56,1.85,3.439909,-2.24,2.2,0.083333,0.233333,0
184,https://www.nhl.com/gamecenter/2007020185,2007/11/02,Dallas Stars,Coyotes,-1.47,1.460398,0.251726,-2.41,1.992685,0.476706,...,3.85,-7.0,-3.73,-4.03,3.169516,0.67,-0.5,0.016667,0.200000,0
185,https://www.nhl.com/gamecenter/2007020186,2007/11/02,Edmonton Oilers,Nashville Predators,-2.55,-0.427064,0.131494,-3.35,-0.461974,0.301930,...,-0.55,8.0,-2.71,-2.61,-0.533771,-0.01,1.2,0.020979,0.440559,0
186,https://www.nhl.com/gamecenter/2007020187,2007/11/02,San Jose Sharks,Los Angeles Kings,1.60,-0.830282,1.070527,2.45,-0.916607,1.017180,...,0.36,-7.0,1.98,1.99,-1.096545,1.45,6.1,0.038462,0.153846,0
187,https://www.nhl.com/gamecenter/2007020188,2007/11/03,Montréal Canadiens,Toronto Maple Leafs,1.06,-0.457981,2.341007,-0.36,-0.038425,2.946117,...,2.83,33.0,3.02,3.57,-0.406343,3.23,1.1,0.198718,0.244755,0
188,https://www.nhl.com/gamecenter/2007020189,2007/11/03,Ottawa Senators,Boston Bruins,2.66,0.833424,1.068271,3.21,1.333901,1.120448,...,2.91,-6.0,4.11,4.62,0.807193,2.10,2.9,0.272727,0.404040,1
189,https://www.nhl.com/gamecenter/2007020190,2007/11/03,New York Islanders,Pittsburgh Penguins,1.47,0.053590,-0.599915,1.65,0.577136,-1.393353,...,-1.69,-15.0,2.92,3.35,-0.028653,-1.66,0.2,0.100000,0.111111,1
190,https://www.nhl.com/gamecenter/2007020191,2007/11/03,New York Rangers,New Jersey Devils,1.12,-1.887839,3.019908,3.60,-2.535917,3.420050,...,0.88,1.0,4.48,5.24,-3.593985,4.48,3.5,0.053030,0.528926,1


In [36]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, random_state=42, stratify=y_all)

In [37]:
# Taken from Siraj Raval
#for measuring training time#for me 
from time import time 
# F1 score (also F-score or F-measure) is a measure of a test's accuracy. 
#It considers both the precision p and the recall r of the test to compute 
#the score: p is the number of correct positive results divided by the number of 
#all positive results, and r is the number of correct positive results divided by 
#the number of positive results that should have been returned. The F1 score can be 
#interpreted as a weighted average of the precision and recall, where an F1 score 
#reaches its best value at 1 and worst at 0.
from sklearn.metrics import f1_score

def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print("Trained model in {:.4f} seconds".format(end - start))

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    
    end = time()
    # Print and return results
    print("Made predictions in {:.4f} seconds.".format(end - start))
    
    return f1_score(target, y_pred), sum(target == y_pred) / float(len(y_pred))


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    f1, acc = predict_labels(clf, X_train, y_train)
    print(f1, acc)
    print("F1 score and accuracy score for training set: {:.4f} , {:.4f}.".format(f1 , acc))
    
    f1, acc = predict_labels(clf, X_test, y_test)
    print("F1 score and accuracy score for test set: {:.4f} , {:.4f}.".format(f1 , acc))
    
def train_predictproba(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    f1, acc = predict_labels(clf, X_train, y_train)
    print(f1, acc)
    print("F1 score and accuracy score for training set: {:.4f} , {:.4f}.".format(f1 , acc))
    
    display(clf.predict_proba(X_test))
#     print("F1 score and accuracy score for test set: {:.4f} , {:.4f}.".format(f1 , acc))

In [38]:
clf_A = LogisticRegression(random_state = 42)
clf_B = SVC(random_state = 43, kernel = 'rbf')
clf_C = xgb.XGBClassifier(seed = 44)
clf_D = SVC(probability=True)

train_predict(clf_A, x_train, y_train, x_test, y_test)
print('')
train_predict(clf_B, x_train, y_train, x_test, y_test)
print('')
train_predict(clf_C, x_train, y_train, x_test, y_test)
print('')
train_predict(clf_D, x_train, y_train, x_test, y_test)
print('')


Training a LogisticRegression using a training set size of 7405. . .
Trained model in 0.0341 seconds
Made predictions in 0.0019 seconds.
0.6639839034205232 0.5715057393652937
F1 score and accuracy score for training set: 0.6640 , 0.5715.
Made predictions in 0.0005 seconds.
F1 score and accuracy score for test set: 0.6553 , 0.5621.

Training a SVC using a training set size of 7405. . .
Trained model in 2.7647 seconds
Made predictions in 1.2740 seconds.
0.6886128364389233 0.5937879810938556
F1 score and accuracy score for training set: 0.6886 , 0.5938.
Made predictions in 0.3222 seconds.
F1 score and accuracy score for test set: 0.6749 , 0.5734.

Training a XGBClassifier using a training set size of 7405. . .
Trained model in 0.6775 seconds
Made predictions in 0.0227 seconds.
0.7111302661351148 0.6379473328831871
F1 score and accuracy score for training set: 0.7111 , 0.6379.
Made predictions in 0.0051 seconds.
F1 score and accuracy score for test set: 0.6632 , 0.5778.

Training a SVC usi

/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Trained model in 11.5067 seconds
Made predictions in 1.2780 seconds.
0.6886128364389233 0.5937879810938556
F1 score and accuracy score for training set: 0.6886 , 0.5938.
Made predictions in 0.3194 seconds.
F1 score and accuracy score for test set: 0.6749 , 0.5734.



In [39]:
from sklearn import metrics

predictProba = clf_D.predict_proba(newx_all)
print(clf_D.classes_)
display(predictProba)

newGamesFrame['0_Confidence'] = predictProba[:,0]
newGamesFrame['1_Confidence'] = predictProba[:,1]

predictions = clf_B.predict(newx_all)
print(f1_score(newy_all, predictions), sum(newy_all == predictions) / float(len(predictions)))
# predictDf = pd.DataFrame(predictions, columns=['prediction'])
newGamesFrame['Prediction'] = predictions

newGamesFrame

[0 1]


array([[0.4625617 , 0.5374383 ],
       [0.54165961, 0.45834039],
       [0.43064588, 0.56935412],
       ...,
       [0.40587016, 0.59412984],
       [0.40384005, 0.59615995],
       [0.40490421, 0.59509579]])

0.6934256055363321 0.5920810313075506


,ID,Date,HomeTeam,AwayTeam,CF%,CSh%,CSv%,FF%,FSh%,FSv%,...,SDiff,Sh%,Sv,FOW%,W%,FavoritesW%,Result,0_Confidence,1_Confidence,Prediction
11255,https://www.nhl.com/gamecenter/2017020186,2017/11/02,Boston Bruins,Vegas Golden Knights,-1.12,-0.778916,-0.690236,-0.26,-1.321017,-0.902049,...,1.68,-2.028334,-1.75,-0.8,-0.327273,0.257362,1,0.462562,0.537438,1
11256,https://www.nhl.com/gamecenter/2017020187,2017/11/02,Washington Capitals,New York Islanders,-1.85,-1.019858,-0.441613,-2.09,-1.575384,-0.341264,...,-1.31,-2.083285,-0.94,3.0,-0.166667,0.368909,1,0.541660,0.458340,0
11257,https://www.nhl.com/gamecenter/2017020188,2017/11/02,Ottawa Senators,Detroit Red Wings,-1.80,1.591117,0.092414,1.01,2.044135,-0.612546,...,3.10,2.640459,-1.18,6.7,-0.044872,0.321633,1,0.430646,0.569354,1
11258,https://www.nhl.com/gamecenter/2017020189,2017/11/02,Tampa Bay Lightning,New York Rangers,-0.55,1.602127,1.686500,-0.19,1.966037,2.210855,...,-1.36,3.366109,2.87,-2.8,0.461538,0.340786,0,0.402705,0.597295,1
11259,https://www.nhl.com/gamecenter/2017020190,2017/11/02,Florida Panthers,Columbus Blue Jackets,-4.61,1.404778,-1.736500,-4.64,1.668242,-2.145501,...,-7.38,2.260701,-2.58,5.9,-0.303030,0.340977,0,0.548868,0.451132,0
11260,https://www.nhl.com/gamecenter/2017020191,2017/11/02,St. Louis Blues,Philadelphia Flyers,-1.48,0.322340,1.266298,-0.79,0.257881,1.725119,...,-3.14,0.559013,2.81,-3.1,0.269231,0.334474,0,0.408040,0.591960,1
11261,https://www.nhl.com/gamecenter/2017020192,2017/11/02,Minnesota Wild,Montréal Canadiens,-7.28,2.199241,1.702287,-6.66,2.698675,2.354117,...,-9.01,3.754386,3.07,5.0,0.066667,0.318442,1,0.439287,0.560713,1
11262,https://www.nhl.com/gamecenter/2017020193,2017/11/02,Winnipeg Jets,Dallas Stars,-7.27,1.279969,0.248114,-8.68,1.990895,0.438289,...,-8.88,2.158308,0.53,-1.0,-0.037879,0.316555,1,0.517838,0.482162,0
11263,https://www.nhl.com/gamecenter/2017020194,2017/11/02,Colorado Avalanche,Carolina Hurricanes,-8.05,1.788203,0.112443,-6.16,1.828895,0.231109,...,-3.70,1.848007,-0.16,-7.4,0.145455,0.342385,1,0.465360,0.534640,1
11264,https://www.nhl.com/gamecenter/2017020195,2017/11/02,Calgary Flames,Pittsburgh Penguins,-0.02,-0.918719,1.662550,-1.97,-1.028084,2.492625,...,-4.14,-0.846410,3.19,-2.2,-0.038462,0.348636,1,0.451778,0.548222,1


In [40]:
confusionMatrix = metrics.confusion_matrix(newy_all, predictions)
display(confusionMatrix)

tn = confusionMatrix[0][0]
fp = confusionMatrix[0][1]
fn = confusionMatrix[1][0]
tp = confusionMatrix[1][1]
actualYes = fn + tp 
actualNo = tn + fp
predictedYes = fp + tp
predictedNo = tn + fn

print('When home team wins, classifier predicts they will win %6.2f%% of the time' % (tp / actualYes * 100))
print('When home team loses, classifier predicts they will win %6.2f%% of the time' % (fp / actualNo * 100))
print('When home team loses, classifier predicts they will lose %6.2f%% of the time' % (tn / actualNo * 100))
print('When classifer predicts home team will win, home team actually wins %6.2f%% of the time' % (tp / predictedYes * 100))
print('When classifer predicts home team will lose, home team actually loses %6.2f%% of the time' % (tn / predictedNo * 100))

array([[142, 325],
       [118, 501]])

When home team wins, classifier predicts they will win  80.94% of the time
When home team loses, classifier predicts they will win  69.59% of the time
When home team loses, classifier predicts they will lose  30.41% of the time
When classifer predicts home team will win, home team actually wins  60.65% of the time
When classifer predicts home team will lose, home team actually loses  54.62% of the time


In [41]:
oddsFrame = pd.read_csv(r'/Users/Milton/Developer/Hockey/Article Testing Sandbox/Odds 2007-2018 Raw.csv')

oddsFrame.loc[(oddsFrame['HomeTeam'] == 'New York Islanders') & (oddsFrame['Date'] == '2018/01/01')]

,Date,HomeTeam,AwayTeam,HomeOdds,AwayOdds


In [43]:
oddsFrame = pd.read_csv(r'/Users/Milton/Developer/Hockey/Article Testing Sandbox/Odds 2007-2018 Raw.csv')

def findPayout(date, homeTeam, awayTeam):
    for index, row in oddsFrame.iterrows():
        if date == row['Date'] and row['HomeTeam'] in homeTeam and row['AwayTeam'] in awayTeam:
            return row['HomeOdds'], row['AwayOdds']
    print('Could not find')
    print(date)
    print(homeTeam)
    print(awayTeam)

def calculateBetWith100Outcome(prediction, result, homePayout, awayPayout):
    if result == 1 and prediction == 1:
        # Predicted home win correctly
        return homePayout * 100
    elif result == 0 and prediction == 0:
        # Predicted lose correctly
        return awayPayout * 100
    else:
        # Predicted wrong
        return  -100
    
def calculateBetFor100TotalInvestment(prediction, result, homePayout, awayPayout):
    if prediction == 1:
        # Predicted correctly
        return abs(100 / homePayout)
    elif prediction == 0:
        return abs(100 / awayPayout)
    
def calculateBetFor100Outcome(prediction, result, homePayout, awayPayout):
    if result == prediction:
        # Predicted correctly
        return 100
    elif prediction == 0:
        # Predicted lose incorrectly
        return -100 / awayPayout
    elif prediction == 1:
        # Predicted win incorrectly
        return  -100 / homePayout
    
def calculateBetEvenOutcome(prediction, result, homePayout, awayPayout):
    if result == prediction:
    # Predicted home win correctly
        return 100
    else:
    # Predicted wrong
        return  -100
    
def addOdds(row):
    homePayout, awayPayout = findPayout(row['Date'], row['HomeTeam'], row['AwayTeam'])
    vegasPrediction = None
    if homePayout < awayPayout:
        vegasPrediction = 1
    elif homePayout > awayPayout:
        vegasPrediction = 0
    else:
        vegasPrediction = randint(0, 1)
    row['VegasPrediction'] = vegasPrediction
    row['HomePayout'] = homePayout
    row['AwayPayout'] = awayPayout
    
    row['BetEvenOutcome'] = calculateBetEvenOutcome(row['Prediction'], row['Result'], homePayout, awayPayout)
    
    row['BetWith100Outcome'] = calculateBetWith100Outcome(row['Prediction'], row['Result'], homePayout, awayPayout)
    row['BetFor100Outcome'] = calculateBetFor100Outcome(row['Prediction'], row['Result'], homePayout, awayPayout)
    row['BetFor100TotalInvestment'] = calculateBetFor100TotalInvestment(row['Prediction'], row['Result'], homePayout, awayPayout)
    row['VegasBetWith100Outcome'] = calculateBetWith100Outcome(row['VegasPrediction'], row['Result'], homePayout, awayPayout)
    row['VegasBetFor100Outcome'] = calculateBetFor100Outcome(row['VegasPrediction'], row['Result'], homePayout, awayPayout)
    return row
    
newGamesFrame = newGamesFrame.progress_apply(addOdds, axis=1)

display(newGamesFrame)
newGamesFrame.to_csv('Favorites 2017-2018 Bet Results.csv')


100%|██████████| 1086/1086 [08:32<00:00,  1.55it/s]


,ID,Date,HomeTeam,AwayTeam,CF%,CSh%,CSv%,FF%,FSh%,FSv%,...,Prediction,VegasPrediction,HomePayout,AwayPayout,BetEvenOutcome,BetWith100Outcome,BetFor100Outcome,BetFor100TotalInvestment,VegasBetWith100Outcome,VegasBetFor100Outcome
11255,https://www.nhl.com/gamecenter/2017020186,2017/11/02,Boston Bruins,Vegas Golden Knights,-1.12,-0.778916,-0.690236,-0.26,-1.321017,-0.902049,...,1,1,0.507614,1.770000,100,50.761421,100.000000,197.000000,50.761421,100.0
11256,https://www.nhl.com/gamecenter/2017020187,2017/11/02,Washington Capitals,New York Islanders,-1.85,-1.019858,-0.441613,-2.09,-1.575384,-0.341264,...,0,1,0.751880,1.200000,-100,-100.000000,-83.333333,83.333333,75.187970,100.0
11257,https://www.nhl.com/gamecenter/2017020188,2017/11/02,Ottawa Senators,Detroit Red Wings,-1.80,1.591117,0.092414,1.01,2.044135,-0.612546,...,1,1,0.694444,1.300000,100,69.444444,100.000000,144.000000,69.444444,100.0
11258,https://www.nhl.com/gamecenter/2017020189,2017/11/02,Tampa Bay Lightning,New York Rangers,-0.55,1.602127,1.686500,-0.19,1.966037,2.210855,...,1,1,0.602410,1.500000,-100,-100.000000,-166.000000,166.000000,-100.000000,-166.0
11259,https://www.nhl.com/gamecenter/2017020190,2017/11/02,Florida Panthers,Columbus Blue Jackets,-4.61,1.404778,-1.736500,-4.64,1.668242,-2.145501,...,0,0,0.980392,0.925926,100,92.592593,100.000000,108.000000,92.592593,100.0
11260,https://www.nhl.com/gamecenter/2017020191,2017/11/02,St. Louis Blues,Philadelphia Flyers,-1.48,0.322340,1.266298,-0.79,0.257881,1.725119,...,1,1,0.540541,1.660000,-100,-100.000000,-185.000000,185.000000,-100.000000,-185.0
11261,https://www.nhl.com/gamecenter/2017020192,2017/11/02,Minnesota Wild,Montréal Canadiens,-7.28,2.199241,1.702287,-6.66,2.698675,2.354117,...,1,1,0.854701,1.060000,100,85.470085,100.000000,117.000000,85.470085,100.0
11262,https://www.nhl.com/gamecenter/2017020193,2017/11/02,Winnipeg Jets,Dallas Stars,-7.27,1.279969,0.248114,-8.68,1.990895,0.438289,...,0,1,0.934579,0.970874,-100,-100.000000,-103.000000,103.000000,93.457944,100.0
11263,https://www.nhl.com/gamecenter/2017020194,2017/11/02,Colorado Avalanche,Carolina Hurricanes,-8.05,1.788203,0.112443,-6.16,1.828895,0.231109,...,1,0,1.110000,0.813008,100,111.000000,100.000000,90.090090,-100.000000,-123.0
11264,https://www.nhl.com/gamecenter/2017020195,2017/11/02,Calgary Flames,Pittsburgh Penguins,-0.02,-0.918719,1.662550,-1.97,-1.028084,2.492625,...,1,1,0.694444,1.300000,100,69.444444,100.000000,144.000000,69.444444,100.0


In [18]:
#display(newGamesFrame)#[['Prediction', 'Result', 'HomePayout', 'AwayPayout', 'BetFor100Outcome', 'BetFor100TotalInvestment']]
print("Number of times it bets on underdogs: %d" % (newGamesFrame.loc[(newGamesFrame['Prediction'] != newGamesFrame['VegasPrediction'])].shape[0]))
print("Number of times it bets on underdogs and is right: %d" % (newGamesFrame.loc[(newGamesFrame['Prediction'] != newGamesFrame['VegasPrediction']) & (newGamesFrame['Prediction'] == newGamesFrame['Result'])].shape[0]))
print("Total money: $%6.2f" % ((newGamesFrame.loc[(newGamesFrame['Prediction'] != newGamesFrame['VegasPrediction'])]['VegasBetFor100Outcome'].sum())))


Number of times it bets on underdogs: 170
Vegas number of times it bets on underdogs and is right: 76
Total money: $ 10.00


In [19]:
print("If I bet to make $100 per game (%d games), I would make $%6.2f on $%6.2f (ROI: %6.2f%%)" % (newGamesFrame.shape[0], newGamesFrame['BetFor100Outcome'].sum(), newGamesFrame['BetFor100TotalInvestment'].sum(), (newGamesFrame['BetFor100Outcome'].sum() / newGamesFrame['BetFor100TotalInvestment'].sum() * 100)))
print("Vegas If I bet to make $100 per game (%d games), I would make $%6.2f on $%6.2f" % (newGamesFrame.shape[0], newGamesFrame['VegasBetFor100Outcome'].sum(), newGamesFrame.shape[0] * 100))
print("If I bet $100 to make $100 per game (%d games), I would make $%6.2f on $%6.2f" % (newGamesFrame.shape[0], newGamesFrame['BetEvenOutcome'].sum(), newGamesFrame.shape[0] * 100))

If I bet to make $100 per game (1086 games), I would make $2605.10 on $160037.18 (ROI:   1.63%)
Vegas If I bet to make $100 per game (1086 games), I would make $3394.00 on $108600.00
If I bet $100 to make $100 per game (1086 games), I would make $20000.00 on $108600.00


In [20]:
print('Games where Vegas states there is an equal chance of either team winning')
equalOddsGames = newGamesFrame.loc[newGamesFrame['HomePayout'] == newGamesFrame['AwayPayout']]
display(equalOddsGames)

Games where Vegas states there is an equal chance of either team winning


,ID,Date,HomeTeam,AwayTeam,CF%,CSh%,CSv%,FF%,FSh%,FSv%,...,Prediction,VegasPrediction,HomePayout,AwayPayout,BetEvenOutcome,BetWith100Outcome,BetFor100Outcome,BetFor100TotalInvestment,VegasBetWith100Outcome,VegasBetFor100Outcome
11377,https://www.nhl.com/gamecenter/2017020308,2017/11/19,Vegas Golden Knights,Los Angeles Kings,-0.30,0.863397,-1.314822,-0.22,1.366283,-1.954767,...,1,1,0.952381,0.952381,100,95.238095,100.0,105.0,95.238095,100.0
11439,https://www.nhl.com/gamecenter/2017020370,2017/11/28,Philadelphia Flyers,San Jose Sharks,-1.94,0.652408,-1.257872,-2.92,0.765286,-1.366857,...,1,0,0.952381,0.952381,-100,-100.000000,-105.0,105.0,95.238095,100.0
11446,https://www.nhl.com/gamecenter/2017020377,2017/11/29,Boston Bruins,Tampa Bay Lightning,-1.46,-1.248257,-0.749400,-1.39,-1.576881,-1.062136,...,0,0,0.952381,0.952381,-100,-100.000000,-105.0,105.0,-100.000000,-105.0
11459,https://www.nhl.com/gamecenter/2017020390,2017/12/01,New York Rangers,Carolina Hurricanes,-5.69,1.424650,0.084943,-4.38,1.604275,0.075353,...,1,1,0.952381,0.952381,100,95.238095,100.0,105.0,95.238095,100.0
11464,https://www.nhl.com/gamecenter/2017020395,2017/12/01,Colorado Avalanche,New Jersey Devils,0.93,0.154797,-0.381971,0.12,0.543445,-0.578637,...,1,1,0.952381,0.952381,-100,-100.000000,-105.0,105.0,-100.000000,-105.0
11465,https://www.nhl.com/gamecenter/2017020396,2017/12/02,Philadelphia Flyers,Boston Bruins,-0.08,0.059444,-0.073572,-0.25,0.194835,-0.177345,...,1,0,0.952381,0.952381,-100,-100.000000,-105.0,105.0,95.238095,100.0
11626,https://www.nhl.com/gamecenter/2017020557,2017/12/23,San Jose Sharks,Los Angeles Kings,3.56,-0.787682,-0.539650,3.92,-0.834089,-1.007476,...,1,1,0.952381,0.952381,100,95.238095,100.0,105.0,95.238095,100.0
11724,https://www.nhl.com/gamecenter/2017020655,2018/01/09,Ottawa Senators,Chicago Blackhawks,-3.97,0.128502,-0.817276,-1.64,-0.115628,-1.423202,...,0,1,0.952381,0.952381,100,95.238095,100.0,105.0,-100.000000,-105.0
11774,https://www.nhl.com/gamecenter/2017020705,2018/01/19,Florida Panthers,Vegas Golden Knights,-1.91,-0.783326,-0.677722,-2.10,-1.329171,-0.554516,...,0,1,0.952381,0.952381,-100,-100.000000,-105.0,105.0,95.238095,100.0
11853,https://www.nhl.com/gamecenter/2017020784,2018/02/01,Calgary Flames,Tampa Bay Lightning,0.03,-1.372582,-0.108936,-1.01,-1.751073,0.012763,...,0,1,0.952381,0.952381,100,95.238095,100.0,105.0,-100.000000,-105.0


In [51]:
totalGames = newGamesFrame.shape[0]
totalGamesWithFavorites = totalGames - equalOddsGames.shape[0]

In [52]:
print("If I bet $100 per game (%d games), I would make $%6.2f on $%6.2f" % (newGamesFrame.shape[0], newGamesFrame['BetWith100Outcome'].sum(), totalGames * 100))
print("Classifier")
print("     F1 Score: %6.6f" % (f1_score(newy_all, predictions)))
print("     Accuracy: %6.6f   (%d correct / %d games)" % (sum(newy_all == predictions) / float(len(predictions)), sum(newy_all == predictions), len(predictions)))

If I bet $100 per game (1086 games), I would make $1341.10 on $108600.00
Classifier
     F1 Score: 0.693426
     Accuracy: 0.592081   (643 correct / 1086 games)


In [53]:
print("If I bet $100 per game (%d games), I would make $%6.2f on $%6.2f" % (newGamesFrame.shape[0], newGamesFrame['VegasBetWith100Outcome'].sum(), totalGames * 100))
vegasPredictions = newGamesFrame[['VegasPrediction']].values.flatten()
print("Vegas (Favorites)")
print("     F1 Score: %6.6f" % (f1_score(newy_all, vegasPredictions)))
print("     Accuracy: %6.6f   (%d correct / %d games)" % (sum(newy_all == vegasPredictions) / float(len(vegasPredictions)), sum(newy_all == vegasPredictions), len(vegasPredictions)))

If I bet $100 per game (1086 games), I would make $1834.70 on $108600.00
Vegas (Favorites)
     F1 Score: 0.697806
     Accuracy: 0.606814   (659 correct / 1086 games)


In [54]:
samePrediction = newGamesFrame.loc[newGamesFrame['Prediction'] == newGamesFrame['VegasPrediction']].shape[0]
print("My model is able to identify the favourite team %6.2f%% of the time" % (samePrediction / totalGamesWithFavorites * 100))

My model is able to identify the favourite team  85.02% of the time


In [55]:
print('Assuming Vegas has a pretty good model')
print('     the underdogs win %6.2f%% of the time' % (sum(newy_all != vegasPredictions) / totalGamesWithFavorites * 100))
print('     the favorites win %6.2f%% of the time' % (sum(newy_all == vegasPredictions) / totalGamesWithFavorites * 100))

Assuming Vegas has a pretty good model
     the underdogs win  39.72% of the time
     the favorites win  61.30% of the time


In [56]:
header = ['Team', 'Earnings', 'Total Games', 'Total Investment', 'ROI', 'FavoriteWPercent', 'UnderdogWPercent']
rows = []

def teamPerformance(teamName):
    team = newGamesFrame.loc[(newGamesFrame['HomeTeam'] == teamName) | (newGamesFrame['AwayTeam'] ==  teamName)]
    totalGames = team.shape[0]
    earnings = team['BetFor100Outcome'].sum()
    unequalMatchups = team.loc[team['HomePayout'] != team['AwayPayout']].shape[0]
    favoriteWins = team.loc[team['VegasPrediction'] == team['Result']].shape[0]
#     print("If I bet $100 per game (%d games) on %s, I would make $%6.2f on $%6.2f, %6.2f%% ROI" % (totalGames, teamName, earnings, totalGames * 100, (earnings / totalGames)))
    return [teamName, earnings, totalGames, team['BetFor100TotalInvestment'].sum(), np.float64(earnings / team['BetFor100TotalInvestment'].sum() * 100), np.float64(favoriteWins / unequalMatchups), 1 - (np.float64(favoriteWins / unequalMatchups))]

standings = pd.read_csv(r'/Users/Milton/Developer/Hockey/data/Standings NHL/2018-4-8.csv')
standings.loc[standings['Team'] == 'Arizona Coyotes', 'Team'] = 'Coyotes'

for team in standings[['Team']].values.flatten():
    rows.append(teamPerformance(team))

with open('Bets Team Performance.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(header)
    writer.writerows(rows)
    
bets = pd.read_csv(r'/Users/Milton/Developer/Hockey/Article Testing Sandbox/Bets Team Performance.csv')
print('Max ROI: %6.2f%%, Min ROI: %6.2f%%' % (bets['ROI'].max(), bets['ROI'].min()))

Max ROI:  16.02%, Min ROI: -19.27%


In [72]:
newGamesFrame = pd.read_csv(r'/Users/Milton/Developer/Hockey/Article Testing Sandbox/Favorites 2017-2018 Bet Results.csv')

In [81]:
newGamesFrame['ConfidenceDifference'] = abs(newGamesFrame['0_Confidence'] - newGamesFrame['1_Confidence'])
conMax = newGamesFrame['ConfidenceDifference'].max()
conMedian = newGamesFrame['ConfidenceDifference'].median()
conMin = newGamesFrame['ConfidenceDifference'].min()
conMean = newGamesFrame['ConfidenceDifference'].mean()
print('Confidence Difference')
print('     max: %6.2f' % conMax)
print('     min: %6.2f' % conMin)
print('     median: %6.2f' % conMedian)
print('     mean: %6.2f' % conMean)

header = ['Confidence', 'ROI']
rows = []

def calculateAccuracy(threshold):
    confidentFrame = newGamesFrame.loc[newGamesFrame['ConfidenceDifference'] > threshold]
    correctGuesses = confidentFrame.loc[confidentFrame['Result'] == confidentFrame['Prediction']].shape[0]
#     display(confidentFrame[['HomeTeam', 'AwayTeam', 'Prediction', 'VegasPrediction', 'Result', '0_Confidence', '1_Confidence', 'ConfidenceDifference']].sort_values('ConfidenceDifference'))
    print('Threshold: %6.2f, Accuracy: %6.2f%%' % (threshold, (correctGuesses / confidentFrame.shape[0])))
    print("If I bet to make $100 per game (%d games), I would make $%6.2f on $%6.2f" % (confidentFrame.shape[0], confidentFrame['BetFor100Outcome'].sum(), confidentFrame['BetFor100TotalInvestment'].sum()))
    print("ROI: %6.2f" % (confidentFrame['BetFor100Outcome'].sum() / confidentFrame['BetFor100TotalInvestment'].sum() * 100))
    
    samePrediction = confidentFrame.loc[confidentFrame['Prediction'] == confidentFrame['VegasPrediction']].shape[0]
    print("My model is able to identify the favourite team %6.2f%% of the time" % (samePrediction / confidentFrame.shape[0] * 100))
    return [threshold, (confidentFrame['BetFor100Outcome'].sum() / confidentFrame['BetFor100TotalInvestment'].sum() * 100)]

for i in [x / 100 for x in range(0, 27)]:
    rows.append(calculateAccuracy(i))
    
with open('Confidence Performance.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(header)
    writer.writerows(rows)

# calculateAccuracy(0.20)

# print("If I bet to make $100 per game (%d games), I would make $%6.2f on $%6.2f" % (confidentFrame.shape[0], confidentFrame['BetFor100Outcome'].sum(), totalGames * 100))
# print("If I bet $100 per game (%d games), I would make $%6.2f on $%6.2f" % (confidentFrame.shape[0], confidentFrame['BetWith100Outcome'].sum(), totalGames * 100))
# print("If Vegas bet to make $100 per game (%d games), I would make $%6.2f on $%6.2f" % (confidentFrame.shape[0], confidentFrame['VegasBetFor100Outcome'].sum(), totalGames * 100))
# print("If I bet $100 to make $100 per game (%d games), I would make $%6.2f on $%6.2f" % (confidentFrame.shape[0], confidentFrame['BetEvenOutcome'].sum(), totalGames * 100))

Confidence Difference
     max:   0.26
     min:   0.00
     median:   0.15
     mean:   0.14
Threshold:   0.00, Accuracy:   0.59%
If I bet to make $100 per game (1064 games), I would make $2669.44 on $157567.08
ROI:   1.69
My model is able to identify the favourite team  84.68% of the time
Threshold:   0.01, Accuracy:   0.59%
If I bet to make $100 per game (1064 games), I would make $2669.44 on $157567.08
ROI:   1.69
My model is able to identify the favourite team  84.68% of the time
Threshold:   0.02, Accuracy:   0.59%
If I bet to make $100 per game (1031 games), I would make $2010.38 on $153944.38
ROI:   1.31
My model is able to identify the favourite team  85.45% of the time
Threshold:   0.03, Accuracy:   0.60%
If I bet to make $100 per game (1005 games), I would make $2918.76 on $150928.67
ROI:   1.93
My model is able to identify the favourite team  86.17% of the time
Threshold:   0.04, Accuracy:   0.60%
If I bet to make $100 per game (978 games), I would make $3129.73 on $147752.

In [ ]:
header = ['Month', 'Accuracy', 'Earnings', 'Total Games', 'Total Investment', 'ROI', 'FavoriteWPercent', 'UnderdogWPercent']
rows = []

def monthPerformance(monthString):
    date = "/%s/" % monthString
    month = newGamesFrame.loc[newGamesFrame['Date'].str.contains(date)]
    totalGames = month.shape[0]
    earnings = month['BetFor100Outcome'].sum()
    unequalMatchups = month.loc[month['HomePayout'] != month['AwayPayout']].shape[0]
    favoriteWins = month.loc[month['VegasPrediction'] == month['Result']].shape[0]
    accuracy = month.loc[month['Prediction'] == month['Result']].shape[0]
#     print("If I bet $100 per game (%d games) on %s, I would make $%6.2f on $%6.2f, %6.2f%% ROI" % (totalGames, teamName, earnings, totalGames * 100, (earnings / totalGames)))
    return [monthString, np.float64(accuracy / totalGames), earnings, totalGames, month['BetFor100TotalInvestment'].sum(), np.float64(earnings / month['BetFor100TotalInvestment'].sum() * 100), np.float64(favoriteWins / unequalMatchups), 1 - (np.float64(favoriteWins / unequalMatchups))]

months = ['11', '12', '01', '02', '03', '04']
for i in months:
    print(i)
    rows.append(monthPerformance(i))

with open('Bets Month Performance.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(header)
    writer.writerows(rows)

In [76]:
header = ['FavouritesW%', 'ROI']
rows = []

def selectFavorites(threshold):
    favoriteFrame = newGamesFrame.loc[newGamesFrame['FavoritesW%'] > threshold]
    correctGuesses = favoriteFrame.loc[favoriteFrame['Result'] == favoriteFrame['Prediction']].shape[0]
    print('Threshold: %6.2f, Accuracy: %6.2f%%' % (threshold, (correctGuesses / favoriteFrame.shape[0])))
    print("If I bet to make $100 per game (%d games), I would make $%6.2f on $%6.2f" % (favoriteFrame.shape[0], favoriteFrame['BetFor100Outcome'].sum(), favoriteFrame['BetFor100TotalInvestment'].sum()))
    print('ROI: %6.2f%%' % (favoriteFrame['BetFor100Outcome'].sum() / favoriteFrame['BetFor100TotalInvestment'].sum() * 100))
    return [threshold, (favoriteFrame['BetFor100Outcome'].sum() / favoriteFrame['BetFor100TotalInvestment'].sum() * 100)]

for i in [x / 100 for x in range(25, 38, 1)]:
    rows.append(selectFavorites(i))
    
with open('Favourites Performance.csv', 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(header)
    writer.writerows(rows)

Threshold:   0.25, Accuracy:   0.59%
If I bet to make $100 per game (1086 games), I would make $2605.10 on $160037.18
ROI:   1.63%
Threshold:   0.26, Accuracy:   0.59%
If I bet to make $100 per game (1085 games), I would make $2505.10 on $159840.18
ROI:   1.57%
Threshold:   0.27, Accuracy:   0.59%
If I bet to make $100 per game (1081 games), I would make $2318.10 on $159230.18
ROI:   1.46%
Threshold:   0.28, Accuracy:   0.59%
If I bet to make $100 per game (1075 games), I would make $2261.10 on $158266.71
ROI:   1.43%
Threshold:   0.29, Accuracy:   0.59%
If I bet to make $100 per game (1071 games), I would make $2032.53 on $157769.28
ROI:   1.29%
Threshold:   0.30, Accuracy:   0.59%
If I bet to make $100 per game (1056 games), I would make $2459.99 on $155503.49
ROI:   1.58%
Threshold:   0.31, Accuracy:   0.59%
If I bet to make $100 per game (1028 games), I would make $2260.99 on $151158.65
ROI:   1.50%
Threshold:   0.32, Accuracy:   0.60%
If I bet to make $100 per game (932 games), I 

In [74]:
def selectFavoritesAndDifference(favoriteThreshold, confidenceThreshold):
    favoriteFrame = newGamesFrame.loc[(newGamesFrame['FavoritesW%'] > favoriteThreshold) & (newGamesFrame['ConfidenceDifference'] > confidenceThreshold)]
    correctGuesses = favoriteFrame.loc[favoriteFrame['Result'] == favoriteFrame['Prediction']].shape[0]
    print('Favorites Threshold: %6.2f, Confidence Threshold: %6.2f' % (favoriteThreshold, confidenceThreshold))
#     print('Threshold: %6.2f, Accuracy: %6.2f%%' % (threshold, (correctGuesses / favoriteFrame.shape[0])))
    print("If I bet to make $100 per game (%d games), I would make $%6.2f on $%6.2f" % (favoriteFrame.shape[0], favoriteFrame['BetFor100Outcome'].sum(), favoriteFrame['BetFor100TotalInvestment'].sum()))
    print('ROI: %6.2f%%' % (favoriteFrame['BetFor100Outcome'].sum() / favoriteFrame['BetFor100TotalInvestment'].sum() * 100))
    
for confidence in [x / 100 for x in range(0, 28)]:
    for favorite in [x / 100 for x in range(25, 40, 1)]:
        selectFavoritesAndDifference(favorite, confidence)

Favorites Threshold:   0.25, Confidence Threshold:   0.00
If I bet to make $100 per game (1064 games), I would make $2669.44 on $157567.08
ROI:   1.69%
Favorites Threshold:   0.26, Confidence Threshold:   0.00
If I bet to make $100 per game (1063 games), I would make $2569.44 on $157370.08
ROI:   1.63%
Favorites Threshold:   0.27, Confidence Threshold:   0.00
If I bet to make $100 per game (1059 games), I would make $2382.44 on $156760.08
ROI:   1.52%
Favorites Threshold:   0.28, Confidence Threshold:   0.00
If I bet to make $100 per game (1053 games), I would make $2325.44 on $155796.61
ROI:   1.49%
Favorites Threshold:   0.29, Confidence Threshold:   0.00
If I bet to make $100 per game (1049 games), I would make $2096.87 on $155299.18
ROI:   1.35%
Favorites Threshold:   0.30, Confidence Threshold:   0.00
If I bet to make $100 per game (1035 games), I would make $2424.33 on $153133.39
ROI:   1.58%
Favorites Threshold:   0.31, Confidence Threshold:   0.00
If I bet to make $100 per game

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in double_scalars
  import sys



If I bet to make $100 per game (31 games), I would make $-206.57 on $5069.10
ROI:  -4.08%
Favorites Threshold:   0.38, Confidence Threshold:   0.03
If I bet to make $100 per game (5 games), I would make $-212.57 on $769.57
ROI: -27.62%
Favorites Threshold:   0.39, Confidence Threshold:   0.03
If I bet to make $100 per game (0 games), I would make $  0.00 on $  0.00
ROI:    nan%
Favorites Threshold:   0.25, Confidence Threshold:   0.04
If I bet to make $100 per game (978 games), I would make $3129.73 on $147752.69
ROI:   2.12%
Favorites Threshold:   0.26, Confidence Threshold:   0.04
If I bet to make $100 per game (977 games), I would make $3029.73 on $147555.69
ROI:   2.05%
Favorites Threshold:   0.27, Confidence Threshold:   0.04
If I bet to make $100 per game (975 games), I would make $3042.73 on $147217.69
ROI:   2.07%
Favorites Threshold:   0.28, Confidence Threshold:   0.04
If I bet to make $100 per game (969 games), I would make $2985.73 on $146254.22
ROI:   2.04%
Favorites Thre

Favorites Threshold:   0.36, Confidence Threshold:   0.07
If I bet to make $100 per game (85 games), I would make $ 15.46 on $13246.44
ROI:   0.12%
Favorites Threshold:   0.37, Confidence Threshold:   0.07
If I bet to make $100 per game (28 games), I would make $-86.57 on $4726.10
ROI:  -1.83%
Favorites Threshold:   0.38, Confidence Threshold:   0.07
If I bet to make $100 per game (5 games), I would make $-212.57 on $769.57
ROI: -27.62%
Favorites Threshold:   0.39, Confidence Threshold:   0.07
If I bet to make $100 per game (0 games), I would make $  0.00 on $  0.00
ROI:    nan%
Favorites Threshold:   0.25, Confidence Threshold:   0.08
If I bet to make $100 per game (850 games), I would make $2397.19 on $132716.25
ROI:   1.81%
Favorites Threshold:   0.26, Confidence Threshold:   0.08
If I bet to make $100 per game (850 games), I would make $2397.19 on $132716.25
ROI:   1.81%
Favorites Threshold:   0.27, Confidence Threshold:   0.08
If I bet to make $100 per game (848 games), I would ma

Favorites Threshold:   0.27, Confidence Threshold:   0.12
If I bet to make $100 per game (711 games), I would make $1115.78 on $114675.94
ROI:   0.97%
Favorites Threshold:   0.28, Confidence Threshold:   0.12
If I bet to make $100 per game (708 games), I would make $1358.78 on $114137.94
ROI:   1.19%
Favorites Threshold:   0.29, Confidence Threshold:   0.12
If I bet to make $100 per game (705 games), I would make $1058.78 on $113711.94
ROI:   0.93%
Favorites Threshold:   0.30, Confidence Threshold:   0.12
If I bet to make $100 per game (696 games), I would make $1215.78 on $112115.94
ROI:   1.08%
Favorites Threshold:   0.31, Confidence Threshold:   0.12
If I bet to make $100 per game (677 games), I would make $1007.78 on $108840.44
ROI:   0.93%
Favorites Threshold:   0.32, Confidence Threshold:   0.12
If I bet to make $100 per game (614 games), I would make $2179.78 on $99096.44
ROI:   2.20%
Favorites Threshold:   0.33, Confidence Threshold:   0.12
If I bet to make $100 per game (481 g

If I bet to make $100 per game (503 games), I would make $-853.27 on $85807.89
ROI:  -0.99%
Favorites Threshold:   0.30, Confidence Threshold:   0.16
If I bet to make $100 per game (495 games), I would make $-810.27 on $84325.89
ROI:  -0.96%
Favorites Threshold:   0.31, Confidence Threshold:   0.16
If I bet to make $100 per game (482 games), I would make $-920.27 on $81958.40
ROI:  -1.12%
Favorites Threshold:   0.32, Confidence Threshold:   0.16
If I bet to make $100 per game (439 games), I would make $840.73 on $75025.40
ROI:   1.12%
Favorites Threshold:   0.33, Confidence Threshold:   0.16
If I bet to make $100 per game (346 games), I would make $-181.27 on $59373.36
ROI:  -0.31%
Favorites Threshold:   0.34, Confidence Threshold:   0.16
If I bet to make $100 per game (217 games), I would make $-1036.34 on $37896.43
ROI:  -2.73%
Favorites Threshold:   0.35, Confidence Threshold:   0.16
If I bet to make $100 per game (111 games), I would make $-823.00 on $19789.00
ROI:  -4.16%
Favorite

ROI:  -5.15%
Favorites Threshold:   0.30, Confidence Threshold:   0.20
If I bet to make $100 per game (84 games), I would make $-1064.00 on $14292.09
ROI:  -7.44%
Favorites Threshold:   0.31, Confidence Threshold:   0.20
If I bet to make $100 per game (84 games), I would make $-1064.00 on $14292.09
ROI:  -7.44%
Favorites Threshold:   0.32, Confidence Threshold:   0.20
If I bet to make $100 per game (76 games), I would make $-1102.00 on $13074.09
ROI:  -8.43%
Favorites Threshold:   0.33, Confidence Threshold:   0.20
If I bet to make $100 per game (64 games), I would make $-1774.00 on $11007.09
ROI: -16.12%
Favorites Threshold:   0.34, Confidence Threshold:   0.20
If I bet to make $100 per game (48 games), I would make $-2031.00 on $8321.09
ROI: -24.41%
Favorites Threshold:   0.35, Confidence Threshold:   0.20
If I bet to make $100 per game (20 games), I would make $-905.00 on $3707.00
ROI: -24.41%
Favorites Threshold:   0.36, Confidence Threshold:   0.20
If I bet to make $100 per game (

ROI: -100.00%
Favorites Threshold:   0.37, Confidence Threshold:   0.24
If I bet to make $100 per game (1 games), I would make $-145.00 on $145.00
ROI: -100.00%
Favorites Threshold:   0.38, Confidence Threshold:   0.24
If I bet to make $100 per game (0 games), I would make $  0.00 on $  0.00
ROI:    nan%
Favorites Threshold:   0.39, Confidence Threshold:   0.24
If I bet to make $100 per game (0 games), I would make $  0.00 on $  0.00
ROI:    nan%
Favorites Threshold:   0.25, Confidence Threshold:   0.25
If I bet to make $100 per game (1 games), I would make $-145.00 on $145.00
ROI: -100.00%
Favorites Threshold:   0.26, Confidence Threshold:   0.25
If I bet to make $100 per game (1 games), I would make $-145.00 on $145.00
ROI: -100.00%
Favorites Threshold:   0.27, Confidence Threshold:   0.25
If I bet to make $100 per game (1 games), I would make $-145.00 on $145.00
ROI: -100.00%
Favorites Threshold:   0.28, Confidence Threshold:   0.25
If I bet to make $100 per game (1 games), I would 

In [45]:
newGamesFrame['FavoritesW%'].max()

0.3869154795709335

In [50]:
newGamesFrame['ImpliedProbabilityHome'] = 100 / (100 + 100 * newGamesFrame['HomePayout'])  
newGamesFrame['ImpliedProbabilityAway'] = 100 / (100 + 100 * newGamesFrame['AwayPayout'])  

valueFrame = newGamesFrame.loc[((newGamesFrame['1_Confidence'] > newGamesFrame['ImpliedProbabilityHome']) & (newGamesFrame['1_Confidence'] > newGamesFrame['0_Confidence'])) | ((newGamesFrame['0_Confidence'] > newGamesFrame['ImpliedProbabilityAway']) & (newGamesFrame['1_Confidence'] < newGamesFrame['0_Confidence']))]

print("If I bet $100 per game (%d games), I would make $%6.2f on $%6.2f" % (valueFrame.shape[0], valueFrame['BetWith100Outcome'].sum(), valueFrame.shape[0] * 100))
print("     Accuracy: %6.6f   (%d correct / %d games)" % (sum(valueFrame['Result'] == valueFrame['Prediction']) / float(len(valueFrame)), sum(valueFrame['Result'] == valueFrame['Prediction']), len(valueFrame)))
print("If I bet to make $100 per game (%d games), I would make $%6.2f on $%6.2f" % (valueFrame.shape[0], valueFrame['BetFor100Outcome'].sum(), valueFrame.shape[0] * 100))

valueFrame[['Result', 'Prediction', '0_Confidence', '1_Confidence', 'ImpliedProbabilityHome', 'ImpliedProbabilityAway']]

If I bet $100 per game (428 games), I would make $-960.56 on $42800.00
     Accuracy: 0.514019   (220 correct / 428 games)
If I bet to make $100 per game (428 games), I would make $-779.57 on $42800.00


,Result,Prediction,0_Confidence,1_Confidence,ImpliedProbabilityHome,ImpliedProbabilityAway
11256,1,0,0.541660,0.458340,0.570815,0.454545
11259,0,0,0.548868,0.451132,0.504950,0.519231
11261,1,1,0.439287,0.560713,0.539171,0.485437
11262,1,0,0.517838,0.482162,0.516908,0.507389
11263,1,1,0.465360,0.534640,0.473934,0.551570
11265,0,0,0.517669,0.482331,0.539171,0.485437
11266,1,0,0.513030,0.486970,0.579832,0.444444
11268,0,1,0.445803,0.554197,0.500000,0.523810
11270,0,1,0.406970,0.593030,0.537037,0.487805
11272,0,1,0.444197,0.555803,0.539171,0.485437


In [69]:
newGamesFrame['0_Confidence']

KeyError: '0_Confidence'